**PART 5: Model Deployment & Cloud MLOps**

In [6]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

ml_client = MLClient(
    DefaultAzureCredential(),
    subscription_id="26c26559-208a-4c64-a94f-f0e8e4ab3b2f",
    resource_group_name="florida-housing-ml-ws-westus3",
    workspace_name="florida-housing-workspace-westus3"
)

data_asset = Data(
    path="homes.csv",
    type=AssetTypes.URI_FILE,
    description="Raw homes dataset for Florida housing project",
    name="homes"
)

ml_client.data.create_or_update(data_asset)
print("Dataset registered successfully.")

Dataset registered successfully.


In [7]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
import pandas as pd

# Connect to workspace
ml_client = MLClient(
    DefaultAzureCredential(),
    subscription_id="26c26559-208a-4c64-a94f-f0e8e4ab3b2f",
    resource_group_name="florida-housing-ml-ws-westus3",
    workspace_name="florida-housing-workspace-westus3"

)

# Retrieve your FILE data asset
data_asset = ml_client.data.get(name="homes", version="1")

# FILE assets in Azure ML v2 give a direct local path
csv_path = data_asset.path
print("CSV Path:", csv_path)

# Load CSV into pandas
df = pd.read_csv(csv_path)
df.head()

CSV Path: azureml://subscriptions/26c26559-208a-4c64-a94f-f0e8e4ab3b2f/resourcegroups/florida-housing-ml-ws-westus3/workspaces/florida-housing-workspace-westus3/datastores/workspaceblobstore/paths/LocalUpload/822cc83f82ef3dca18094f6b5a7c4b2c54e780ffa09d51974820dc5d31009cb1/homes.csv


/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/mlflow/__init__.py:41: UserWarning: Versions of mlflow (3.1.1) and mlflow-skinny (2.22.1) are different. This may lead to unexpected behavior. Please install the same version of both packages.
  mlflow.mismatch._check_version_mismatch()


,Folio,PIN,PropertyType,Owner1,Owner2,MailingAddress1,MailingAddress2,MailingCity,MailingState,MailingZip,...,TotalHeatedAreaSqFt,TotalGrossAreaSqFt,JustValue,AssessedValue,TaxableValue,LastSaleDate,LastSalePrice,VacantImproved,Qualified,random
0,883298300,U-28-30-21-9PC-000110-00003.0,SINGLE FAMILY,FINN JOHN,FINN JERICHO,16410 CHAPMAN CROSSING DR,NaN,LITHIA,FL,33547-3956,...,4747,6513,851774,822318,772318,5/31/2019,889000,Improved,Qualified,5
1,883296182,U-28-30-21-770-EE0003-00004.0,SINGLE FAMILY,GOODHEAD BRIAN W,GOODHEAD MELISSA L,5907 CHURCHSIDE DR,NaN,LITHIA,FL,33547-4889,...,3412,4493,453283,412602,362602,8/12/2015,559000,Improved,Qualified,6
2,883296632,U-28-30-21-771-II0000-00010.0,SINGLE FAMILY,GROUSE MICHAEL J TRUSTEE,GROUSE BONNY R TRUSTEE,15108 KESTRELGLEN WAY,NaN,LITHIA,FL,33547-4820,...,2022,2888,210317,210317,210317,12/13/2017,100,Improved,Unqualified,11
3,883534162,U-32-30-21-69Y-000065-00031.0,SINGLE FAMILY,GORDON GEORGE,NaN,15829 SORAWATER DR,NaN,LITHIA,FL,33547-3911,...,3716,4927,563543,563543,563543,9/19/2019,381900,Improved,Unqualified,27
4,880921070,U-22-30-21-381-000003-00004.0,SINGLE FAMILY,BURGESS MICHAEL,BURGESS GIOVANA,16926 HAWKRIDGE RD,NaN,LITHIA,FL,33547-5809,...,1975,2520,212286,186770,136770,4/20/2015,100,Improved,Unqualified,45


In [5]:

"""
WHY:
We prepare the dataset for modeling.

WHAT:
- Select numeric predictors only
- Train/Val/Test split
"""

import pandas as pd
import numpy as np
import mlflow
import mlflow.sklearn

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso

import matplotlib.pyplot as plt
import seaborn as sns

# ------- Select feature columns (use your engineered df) -------
target = "JustValue"

# remove IDs or text columns
X = df.select_dtypes(include=[np.number]).drop(columns=[target], errors="ignore")
y = df[target]

# Split (70/15/15)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.30, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.50, random_state=42)

print("Train:", X_train.shape, " Val:", X_val.shape, " Test:", X_test.shape)


Train: (5234, 18)  Val: (1122, 18)  Test: (1122, 18)


In [9]:

"""
WHY:
Lasso regression is a linear model with L1 regularization.
It shrinks weak features toward zero and helps identify important predictors.
"""

from sklearn.linear_model import Lasso

# Define model
lasso_model = Lasso(alpha=0.0001, max_iter=5000, random_state=42)

# Fit model
lasso_model.fit(X_train, y_train)

# Predict
lasso_val_pred = lasso_model.predict(X_val)

# Metrics
lasso_mae = mean_absolute_error(y_val, lasso_val_pred)
lasso_rmse = np.sqrt(mean_squared_error(y_val, lasso_val_pred))
lasso_r2 = r2_score(y_val, lasso_val_pred)

print("ADVANCED MODEL 2 — Lasso Regression")
print("MAE:", lasso_mae)
print("RMSE:", lasso_rmse)
print("R²:", lasso_r2)


ADVANCED MODEL 2 — Lasso Regression
MAE: 0.003644747978590766
RMSE: 0.004908986984398413
R²: 0.9999999999999987


In [11]:

import joblib
from azureml.core import Workspace
from azureml.core.model import Model

print("Setting up environment...")

# Load workspace (must have config.json)
ws = Workspace.from_config()

# Load best model (saved from Part 3)
lasso_model = joblib.load("lasso_best_model.pkl")

# Final evaluation metrics from Part 3
test_mae = 0.003854026908487407
test_rmse = 0.005202089719754687
test_r2 = 0.9999999999999984

print("Model loaded successfully.")
print("Test MAE:", test_mae)
print("Test RMSE:", test_rmse)
print("Test R²:", test_r2)


Setting up environment...
Model loaded successfully.
Test MAE: 0.003854026908487407
Test RMSE: 0.005202089719754687
Test R²: 0.9999999999999984


In [7]:
# ================================================================
# PART 5 — STEP 1: MODEL REGISTRATION 
# Register the Best Model (Lasso)
# ================================================================

from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import Model

# ---------------------------------------------------------------
# 1. CONNECT TO YOUR NEW WORKSPACE 
# ---------------------------------------------------------------
subscription_id = "26c26559-208a-4c64-a94f-f0e8e4ab3b2f"
resource_group = "florida-housing-ml-ws-westus3"
workspace = "florida-housing-workspace-westus3"

ml_client = MLClient(
    DefaultAzureCredential(),
    subscription_id,
    resource_group,
    workspace
)

print("Connected to workspace:", workspace)

# ---------------------------------------------------------------
# 2. REGISTER THE MODEL
# ---------------------------------------------------------------

# REPLACE THESE WITH YOUR REAL METRICS FROM PART 3
test_mae = 0.0038540269
test_rmse = 0.0052020897
test_r2 = 0.9999999999999984

# YOUR training feature names
feature_list = [
    "SqFt", "Bedrooms", "Bathrooms", "LotSize", "Garage", "Pool",
    "Age", "HOA_Fee", "ZipCode", "City_encoded", "PropertyType_encoded"
]

model = Model(
    path="lasso_best_model.pkl",
    name="lasso_value_model",
    version="1",   
    description="Lasso regression model for Florida Housing value prediction",
    type="custom_model",
    tags={
        "model_type": "lasso_regression",
        "trained_by": "Sanjana",
        "project": "Florida Housing ML",
    },
    properties={
        "MAE": str(test_mae),
        "RMSE": str(test_rmse),
        "R2": str(test_r2),
        "features_used": ", ".join(feature_list),
        "training_date": "2025-11-19"
    }
)

registered_model = ml_client.models.create_or_update(model)

print("\nMODEL REGISTERED SUCCESSFULLY!")
print("Model Name:", registered_model.name)
print("Model Version:", registered_model.version)


Connected to workspace: florida-housing-workspace-westus3

MODEL REGISTERED SUCCESSFULLY!
Model Name: lasso_value_model
Model Version: 1


In [10]:
from azure.identity import DefaultAzureCredential
from azure.mgmt.resource import ResourceManagementClient

subscription_id = "26c26559-208a-4c64-a94f-f0e8e4ab3b2f"
client = ResourceManagementClient(DefaultAzureCredential(), subscription_id)

providers = [
    "Microsoft.MachineLearningServices",
    "Microsoft.ContainerInstance",
    "Microsoft.ContainerRegistry",
    "Microsoft.Storage",
    "Microsoft.KeyVault",
    "Microsoft.Network"
]

for p in providers:
    rp = client.providers.get(p)
    print(p, "->", rp.registration_state)

Microsoft.MachineLearningServices -> Registered
Microsoft.ContainerInstance -> Registered
Microsoft.ContainerRegistry -> Registered
Microsoft.Storage -> Registered
Microsoft.KeyVault -> Registered
Microsoft.Network -> Registered


In [18]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

subscription_id = "26c26559-208a-4c64-a94f-f0e8e4ab3b2f"
resource_group = "florida-housing-ml-ws-westus3"
workspace = "florida-housing-workspace-westus3"

ml_client = MLClient(
    DefaultAzureCredential(),
    subscription_id,
    resource_group,
    workspace
)

print("Connected to:", workspace)

Connected to: florida-housing-workspace-westus3


In [2]:
from azure.ai.ml import Input

job = ml_client.batch_endpoints.invoke(
    endpoint_name="lasso-batch-003947",   # your batch endpoint
    deployment_name="lasso-deploy",       # your deployment name
    inputs={
        "data": Input(type="uri_file", path="batch_input.csv")
    }
)

print("Job submitted successfully!")
print("Job name:", job.name)

Uploading batch_input.csv (< 1 MB): 100%|██████████| 269/269 [00:00<00:00, 25.0kB/s]




Job submitted successfully!
Job name: batchjob-8f31c840-5e7e-4c7b-8b17-fcc50e4857b5


In [14]:
# List all batch endpoints
endpoints = ml_client.batch_endpoints.list()

print("\n=== Batch Endpoints in Workspace ===\n")
for ep in endpoints:
    print(f"Name: {ep.name}")
    print(f"Description: {ep.description}")
    print(f"Status: {ep.provisioning_state}")
    print("-" * 50)


=== Batch Endpoints in Workspace ===

Name: lasso-batch-003947
Description: Batch endpoint for housing predictions
Status: Succeeded
--------------------------------------------------


In [16]:
deployments = ml_client.batch_deployments.list("lasso-batch-003947")

print("=== Deployments under this endpoint ===")
for dep in deployments:
    print("•", dep.name)

=== Deployments under this endpoint ===
• lasso-deploy


In [15]:
deployment = ml_client.batch_deployments.get(
    endpoint_name="lasso-batch-003947",
    name="lasso-deploy"
)

deployment

BatchDeployment({'provisioning_state': 'Succeeded', 'endpoint_name': 'lasso-batch-003947', 'type': None, 'name': 'lasso-deploy', 'description': None, 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/26c26559-208a-4c64-a94f-f0e8e4ab3b2f/resourceGroups/florida-housing-ml-ws-westus3/providers/Microsoft.MachineLearningServices/workspaces/florida-housing-workspace-westus3/batchEndpoints/lasso-batch-003947/deployments/lasso-deploy', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/ci-westus3/code/Users/patnamsanjana128', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x79c9e1cdee00>, 'serialize': <msrest.serialization.Serializer object at 0x79c9e1cdefb0>, 'model': '/subscriptions/26c26559-208a-4c64-a94f-f0e8e4ab3b2f/resourceGroups/florida-housing-ml-ws-westus3/providers/Microsoft.MachineLearningServices/workspaces/florida-housing-workspace-westus3/models/lasso_value_model/versions/1', 'code_conf

In [23]:
job = ml_client.batch_endpoints.invoke(
    endpoint_name="lasso-batch-003947",
    deployment_name="lasso-deploy",
    inputs={
        "data": Input(type="uri_file", path="batch_input.csv")
    }
)
print(job.name)

batchjob-3d6f3b89-ed2c-4d06-bdda-f28a957d398a


In [34]:
for ep in ml_client.batch_endpoints.list():
    print("Name:", ep.name, "| Status:", ep.provisioning_state)

Name: lasso-batch-003947 | Status: Succeeded


In [35]:
ep = ml_client.batch_endpoints.get("lasso-batch-003947")
print(ep)

auth_mode: aad_token
defaults: {}
description: Batch endpoint for housing predictions
id: /subscriptions/26c26559-208a-4c64-a94f-f0e8e4ab3b2f/resourceGroups/florida-housing-ml-ws-westus3/providers/Microsoft.MachineLearningServices/workspaces/florida-housing-workspace-westus3/batchEndpoints/lasso-batch-003947
location: westus3
name: lasso-batch-003947
properties:
  BatchEndpointCreationApiVersion: '2023-10-01'
  azureml.onlineendpointid: /subscriptions/26c26559-208a-4c64-a94f-f0e8e4ab3b2f/resourceGroups/florida-housing-ml-ws-westus3/providers/Microsoft.MachineLearningServices/workspaces/florida-housing-workspace-westus3/batchEndpoints/lasso-batch-003947
provisioning_state: Succeeded
scoring_uri: https://lasso-batch-003947.westus3.inference.ml.azure.com/jobs
tags: {}



In [44]:
job_status = ml_client.jobs.get(job.name)
print("Status:", job_status.status)

Status: Failed


In [36]:
deployments = ml_client.batch_deployments.list("lasso-batch-003947")
for d in deployments:
    print("Deployment:", d.name, "| Model:", d.model)

Deployment: lasso-deploy | Model: /subscriptions/26c26559-208a-4c64-a94f-f0e8e4ab3b2f/resourceGroups/florida-housing-ml-ws-westus3/providers/Microsoft.MachineLearningServices/workspaces/florida-housing-workspace-westus3/models/lasso_value_model/versions/1


In [43]:
ml_client.jobs.download(name=job.name, download_path="job_logs", all=True)